In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import zipfile

train_df_promt = pd.read_csv('prompts_train.csv')
train_df = pd.read_csv('summaries_train.csv')
test_df_promt = pd.read_csv('prompts_test.csv')
test_df = pd.read_csv('summaries_test.csv')
print(f'Size train set: {train_df.shape} \nSize test set: {test_df.shape}')
print(f'Size train promts: {train_df_promt.shape} \nSize test promts: {test_df_promt.shape}')

import re
import string

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

Size train set: (7165, 5) 
Size test set: (4, 3)
Size train promts: (4, 4) 
Size test promts: (2, 4)


In [2]:
def truncate_text(text, max_length=40):
    """Truncates the string to the specified number of characters"""
    if len(text) > max_length:
        return text[:max_length] + '...'
    return text

train_sample_promt = train_df_promt.head(4).copy()
test_sample_promt = test_df_promt.head(2).copy()
train_sample = train_df.head(4).copy()
test_sample = test_df.head(2).copy()

train_sample_promt.loc[:, 'prompt_text'] = train_sample_promt['prompt_text'].apply(truncate_text)
train_sample_promt.loc[:, 'prompt_question'] = train_sample_promt['prompt_question'].apply(truncate_text)
test_sample_promt.loc[:, 'prompt_text'] = test_sample_promt['prompt_text'].apply(truncate_text)
train_sample.loc[:, 'text'] = train_sample['text'].apply(truncate_text)

In [3]:
df_train = train_df.merge(train_df_promt, on='prompt_id')

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление знаков препинания, специальных символов и чисел
    text = re.sub(r'[^a-z\s]', '', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

stop_words = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then",
    "once", "here", "there", "when", "where", "why", "how", "all", "any",
    "both", "each", "few", "more", "most", "other", "some", "such", "no",
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s",
    "t", "can", "will", "just", "don", "should", "now"
}

def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

from nltk.stem import WordNetLemmatizer

# def lemmatize_text(tokens):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     return lemmatized_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
df_train['text'] = df_train['text'].apply(clean_text)
df_train['text'] = df_train['text'].apply(tokenize_text)
df_train['text'] = df_train['text'].apply(remove_stopwords)
# df_train['text'] = df_train['text'].apply(lemmatize_text)

In [6]:
X = df_train['text']  # Признаки: текст
y_content = df_train['content']  # Целевая переменная 1: content
y_wording = df_train['wording']  # Целевая переменная 2: wording

X_train, X_test, y_train_content, y_test_content, y_train_wording, y_test_wording = train_test_split(
    X, y_content, y_wording, test_size=0.2, random_state=42)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

vectorizer = TfidfVectorizer(max_features=5000)
# Если X_train содержит списки токенов, нужно объединить их обратно в строки
X_train = [' '.join(text) if isinstance(text, list) else text for text in X_train]
X_test = [' '.join(text) if isinstance(text, list) else text for text in X_test]

# Теперь можно преобразовать данные в TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [8]:
X_train_tfidf[:3]

<3x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 93 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# Предположим, что X_train и X_test уже определены ранее
vectorizer = TfidfVectorizer(max_features=5000)

# Если X_train содержит списки токенов, нужно объединить их обратно в строки
X_train = [' '.join(text) if isinstance(text, list) else text for text in X_train]
X_test = [' '.join(text) if isinstance(text, list) else text for text in X_test]

# Преобразование данных в TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Преобразование разреженной матрицы в плотный формат
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

# Масштабирование целей (если необходимо)
scaler = StandardScaler()
y_train_content = scaler.fit_transform(y_train_content.values.reshape(-1, 1)).flatten()
y_test_content = scaler.transform(y_test_content.values.reshape(-1, 1)).flatten()
y_train_wording = scaler.fit_transform(y_train_wording.values.reshape(-1, 1)).flatten()
y_test_wording = scaler.transform(y_test_wording.values.reshape(-1, 1)).flatten()

# Создание модели нейронной сети
model = Sequential()
model.add(Dense(512, input_dim=X_train_dense.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))  # Для предсказания одного значения (content или wording)

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])

# Раннее завершение обучения при отсутствии улучшений
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Обучение модели для метрики 'content'
history_content = model.fit(X_train_dense, y_train_content, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)

# Обучение модели для метрики 'wording'
history_wording = model.fit(X_train_dense, y_train_wording, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping], verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step - loss: 0.7719 - mean_squared_error: 0.7719 - val_loss: 0.3541 - val_mean_squared_error: 0.3541
Epoch 2/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.2364 - mean_squared_error: 0.2364 - val_loss: 0.3367 - val_mean_squared_error: 0.3367
Epoch 3/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.3371 - val_mean_squared_error: 0.3371
Epoch 4/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - loss: 0.0867 - mean_squared_error: 0.0867 - val_loss: 0.3261 - val_mean_squared_error: 0.3261
Epoch 5/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 0.0705 - mean_squared_error: 0.0705 - val_loss: 0.3203 - val_mean_squared_error: 0.3203
Epoch 6/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - loss: 0.0653 - mean_squared_error: 0.0653 - val_loss: 0.3332 - val_mean_squared_error: 0.3332
Epoch 7/100
144/144 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - loss: 0.0579 - mean_squared_error: 0.0579

In [10]:
content_pred = model.predict(X_test_dense)  # Здесь мы используем 'model', а не 'history_content'
wording_pred = model.predict(X_test_dense)

def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse


mcrmse_score_content = calculate_rmse(y_test_content, content_pred)
mcrmse_score_wording = calculate_rmse(y_test_wording, wording_pred)

print(f"MCRMSE = {np.mean([mcrmse_score_content, mcrmse_score_wording])}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MCRMSE = 0.7441264251132851
